In [81]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import re
from time import sleep
import acquire_r as ar

In [2]:
url = 'https://www.pro-football-reference.com/boxscores/202109120nor.htm'
response = requests.get(url)
response

<Response [200]>

- 1 SOUP
- 2 div_grid [0]
- 3 div [0]
- 4 div_wrap [0]
- 5 soup_wrap4 [4]
- 6 soup4 : reparse soup_wrap4
- 7 wx

In [ ]:
#1
soup = BeautifulSoup(response.content, 'html.parser')
#2
div_grid = soup.find_all('div', class_='content_grid')[0]
#3
div = div_grid.find_all('div')[0]
#4
div_wrap = div.find_all('div')[0]
#5
soup_wrap4 = div_wrap.contents[4]
#6
soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
#7
wx = soup4.find_all('td','center')[6]

In [ ]:
wx.contents

In [ ]:
wx.contents[0]

In [ ]:
content = wx.contents[0]

In [ ]:
content = str(content)

In [ ]:
type(content)

## So now we have options...

In [ ]:
# regex to return numerical string at start of object
temp = r'^\d{1,2}'
re.findall(temp,content)

In [ ]:
wind = r'\d{1,2}[\smph]'
# etc....

## Or, a different approach

In [ ]:
wx_digits = r'\d{1,2}'
re.findall(wx_digits,content)

In [ ]:
re.findall(wx_digits,content)[0]

In [ ]:
re.findall(wx_digits,content)[0]

In [ ]:
re.findall(wx_digits,content)[1]

In [ ]:
re.findall(wx_digits,content)[2]

## Ok, now we increase the scale to WEEKLY

In [3]:
url = 'https://www.pro-football-reference.com/years/2022/week_1.htm'

In [4]:
response = requests.get(url)
response

<Response [200]>

In [ ]:
response

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
html_links = soup.find_all('td', class_='right gamelink')

In [7]:
html_links[0].find('a')['href']

'/boxscores/202209080ram.htm'

## List of Games

In [8]:
[item.find('a')['href'] for item in html_links]

['/boxscores/202209080ram.htm',
 '/boxscores/202209110atl.htm',
 '/boxscores/202209110car.htm',
 '/boxscores/202209110chi.htm',
 '/boxscores/202209110cin.htm',
 '/boxscores/202209110det.htm',
 '/boxscores/202209110htx.htm',
 '/boxscores/202209110mia.htm',
 '/boxscores/202209110nyj.htm',
 '/boxscores/202209110was.htm',
 '/boxscores/202209110crd.htm',
 '/boxscores/202209110min.htm',
 '/boxscores/202209110oti.htm',
 '/boxscores/202209110sdg.htm',
 '/boxscores/202209110dal.htm',
 '/boxscores/202209120sea.htm']

In [9]:
games = [item.find('a')['href'] for item in html_links]

In [10]:
games

['/boxscores/202209080ram.htm',
 '/boxscores/202209110atl.htm',
 '/boxscores/202209110car.htm',
 '/boxscores/202209110chi.htm',
 '/boxscores/202209110cin.htm',
 '/boxscores/202209110det.htm',
 '/boxscores/202209110htx.htm',
 '/boxscores/202209110mia.htm',
 '/boxscores/202209110nyj.htm',
 '/boxscores/202209110was.htm',
 '/boxscores/202209110crd.htm',
 '/boxscores/202209110min.htm',
 '/boxscores/202209110oti.htm',
 '/boxscores/202209110sdg.htm',
 '/boxscores/202209110dal.htm',
 '/boxscores/202209120sea.htm']

## List of homes

In [11]:
home_re = r'[a-z]{3}'

In [12]:
home = re.findall(home_re,games[0])[3]

In [13]:
type(home)

str

In [14]:
home

'ram'

In [15]:
homes = [re.findall(home_re,i)[3] for i in games]

In [ ]:
# homes_2 = []
# for i in games:
#     home_re = r'[a-z]{3}'
#     home = re.findall(home_re,i)[3]
#     homes.append(home)

In [16]:
homes

['ram',
 'atl',
 'car',
 'chi',
 'cin',
 'det',
 'htx',
 'mia',
 'nyj',
 'was',
 'crd',
 'min',
 'oti',
 'sdg',
 'dal',
 'sea']

## List of seasons

In [17]:
date_re = r'[0-9]{4}'

In [18]:
re.findall(date_re,games[0])[0]

'2022'

In [19]:
season = re.findall(date_re,games[0])[0]

In [20]:
season

'2022'

In [21]:
seasons = [re.findall(date_re,i)[0] for i in games]

In [22]:
seasons

['2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022']

## List of weeks

In [23]:
weeks = [1 for i in games]

In [24]:
weeks

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

## List of temps

In [25]:
temps = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    temp = re.findall(wx_digits,content)[0]
    temps.append(temp)

In [26]:
temps

['20',
 '20',
 '80',
 '69',
 '76',
 '20',
 '20',
 '92',
 '73',
 '74',
 '20',
 '20',
 '75',
 '20',
 '20',
 '74']

In [ ]:
df = pd.DataFrame({
    'season':seasons,
    'week':weeks,
    'home':homes,
    'temp':temps
})

In [ ]:
df

## List of winds

In [29]:
response

<Response [200]>

In [30]:
winds = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    wind = re.findall(wx_digits,content)[2]
    winds.append(wind)

In [31]:
winds

['73',
 '70',
 '8',
 '16',
 '5',
 '64',
 '69',
 '12',
 '3',
 '4',
 '63',
 '66',
 '9',
 '70',
 '93',
 '6']

## List of hums

In [32]:
response

<Response [200]>

In [33]:
hums = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [34]:
hums

['22',
 '22',
 '81',
 '87',
 '83',
 '22',
 '22',
 '58',
 '79',
 '95',
 '22',
 '22',
 '87',
 '22',
 '22',
 '53']

In [35]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':homes,
    'weather_temperature':temps,
    'weather_wind_mph':winds,
    "weather_humidity":hums
})

In [36]:
df

,schedule_season,schedule_week,team_home,weather_temperature,weather_wind_mph,weather_humidity
0,2022,1,ram,20,73,22
1,2022,1,atl,20,70,22
2,2022,1,car,80,8,81
3,2022,1,chi,69,16,87
4,2022,1,cin,76,5,83
5,2022,1,det,20,64,22
6,2022,1,htx,20,69,22
7,2022,1,mia,92,12,58
8,2022,1,nyj,73,3,79
9,2022,1,was,74,4,95


## Write to .csv

In [37]:
df.to_csv('season_2022_w1.csv', index=False)

---
---
## NEXT WEEK

In [38]:
url = 'https://www.pro-football-reference.com/years/2022/week_2.htm'

In [39]:
response = requests.get(url)

In [40]:
response

<Response [200]>

In [41]:
soup = BeautifulSoup(response.content, 'html.parser')
html_links = soup.find_all('td', class_='right gamelink')
games = [item.find('a')['href'] for item in html_links]

In [42]:
games

['/boxscores/202209150kan.htm',
 '/boxscores/202209180cle.htm',
 '/boxscores/202209180det.htm',
 '/boxscores/202209180jax.htm',
 '/boxscores/202209180nor.htm',
 '/boxscores/202209180nyg.htm',
 '/boxscores/202209180pit.htm',
 '/boxscores/202209180rav.htm',
 '/boxscores/202209180ram.htm',
 '/boxscores/202209180sfo.htm',
 '/boxscores/202209180dal.htm',
 '/boxscores/202209180den.htm',
 '/boxscores/202209180rai.htm',
 '/boxscores/202209180gnb.htm',
 '/boxscores/202209190buf.htm',
 '/boxscores/202209190phi.htm']

## List of homes

In [43]:
home_re = r'[a-z]{3}'
homes = [re.findall(home_re,i)[3] for i in games]

In [44]:
homes

['kan',
 'cle',
 'det',
 'jax',
 'nor',
 'nyg',
 'pit',
 'rav',
 'ram',
 'sfo',
 'dal',
 'den',
 'rai',
 'gnb',
 'buf',
 'phi']

## List of seasons

In [45]:
date_re = r'[0-9]{4}'
seasons = [re.findall(date_re,i)[0] for i in games]

In [46]:
seasons

['2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022']

## List of weeks

In [47]:
weeks = [2 for i in games]

In [48]:
weeks

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

## List of temps

In [49]:
temps = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    temp = re.findall(wx_digits,content)[0]
    temps.append(temp)

In [50]:
temps

['20',
 '80',
 '20',
 '20',
 '20',
 '73',
 '81',
 '82',
 '20',
 '20',
 '20',
 '77',
 '20',
 '73',
 '70',
 '76']

## List of winds

In [51]:
response

<Response [200]>

In [52]:
winds = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    wind = re.findall(wx_digits,content)[2]
    winds.append(wind)

In [53]:
winds

['73',
 '11',
 '57',
 '58',
 '70',
 '12',
 '8',
 '6',
 '71',
 '71',
 '92',
 '11',
 '61',
 '5',
 '11',
 '4']

## List of hums

In [82]:
response

<Response [200]>

In [55]:
hums = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [56]:
hums

['22',
 '55',
 '22',
 '22',
 '22',
 '74',
 '51',
 '61',
 '22',
 '22',
 '22',
 '28',
 '22',
 '76',
 '84',
 '62']

In [57]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':homes,
    'weather_temperature':temps,
    'weather_wind_mph':winds,
    "weather_humidity":hums
})

In [58]:
df

,schedule_season,schedule_week,team_home,weather_temperature,weather_wind_mph,weather_humidity
0,2022,2,kan,20,73,22
1,2022,2,cle,80,11,55
2,2022,2,det,20,57,22
3,2022,2,jax,20,58,22
4,2022,2,nor,20,70,22
5,2022,2,nyg,73,12,74
6,2022,2,pit,81,8,51
7,2022,2,rav,82,6,61
8,2022,2,ram,20,71,22
9,2022,2,sfo,20,71,22


## Write to .csv

In [59]:
df.to_csv('season_2022_w2.csv', index=False)

---
---
## NEXT WEEK with SLEEEEEEEEEEEP......

In [63]:
url = 'https://www.pro-football-reference.com/years/2022/week_3.htm'
response = requests.get(url)

In [64]:
response

<Response [200]>

In [65]:
soup = BeautifulSoup(response.content, 'html.parser')
html_links = soup.find_all('td', class_='right gamelink')
games = [item.find('a')['href'] for item in html_links]
games

['/boxscores/202209220cle.htm',
 '/boxscores/202209250car.htm',
 '/boxscores/202209250chi.htm',
 '/boxscores/202209250clt.htm',
 '/boxscores/202209250mia.htm',
 '/boxscores/202209250min.htm',
 '/boxscores/202209250nwe.htm',
 '/boxscores/202209250nyj.htm',
 '/boxscores/202209250oti.htm',
 '/boxscores/202209250was.htm',
 '/boxscores/202209250sdg.htm',
 '/boxscores/202209250crd.htm',
 '/boxscores/202209250sea.htm',
 '/boxscores/202209250tam.htm',
 '/boxscores/202209250den.htm',
 '/boxscores/202209260nyg.htm']

## List of homes

In [66]:
home_re = r'[a-z]{3}'
homes = [re.findall(home_re,i)[3] for i in games]
homes

['cle',
 'car',
 'chi',
 'clt',
 'mia',
 'min',
 'nwe',
 'nyj',
 'oti',
 'was',
 'sdg',
 'crd',
 'sea',
 'tam',
 'den',
 'nyg']

## List of seasons

In [67]:
date_re = r'[0-9]{4}'
seasons = [re.findall(date_re,i)[0] for i in games]
seasons

['2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022']

## List of weeks

In [68]:
weeks = [3 for i in games]
weeks

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

## List of temps

In [69]:
temps = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    temp = re.findall(wx_digits,content)[0]
    temps.append(temp)

In [70]:
temps

['60',
 '68',
 '66',
 '20',
 '89',
 '20',
 '66',
 '66',
 '77',
 '72',
 '20',
 '20',
 '67',
 '88',
 '73',
 '66']

## List of winds

In [71]:
response

<Response [200]>

In [72]:
winds = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    wind = re.findall(wx_digits,content)[2]
    winds.append(wind)

In [73]:
winds

['21',
 '7',
 '13',
 '65',
 '5',
 '66',
 '10',
 '8',
 '5',
 '9',
 '67',
 '63',
 '6',
 '5',
 '8',
 '5']

## List of hums

In [74]:
response

<Response [200]>

In [75]:
hums = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [76]:
hums

['54',
 '70',
 '68',
 '22',
 '63',
 '22',
 '61',
 '62',
 '72',
 '60',
 '22',
 '22',
 '67',
 '58',
 '21',
 '44']

In [77]:
df = pd.DataFrame({
    'schedule_season':seasons,
    'schedule_week':weeks,
    'team_home':homes,
    'weather_temperature':temps,
    'weather_wind_mph':winds,
    "weather_humidity":hums
})

In [78]:
df

,schedule_season,schedule_week,team_home,weather_temperature,weather_wind_mph,weather_humidity
0,2022,3,cle,60,21,54
1,2022,3,car,68,7,70
2,2022,3,chi,66,13,68
3,2022,3,clt,20,65,22
4,2022,3,mia,89,5,63
5,2022,3,min,20,66,22
6,2022,3,nwe,66,10,61
7,2022,3,nyj,66,8,62
8,2022,3,oti,77,5,72
9,2022,3,was,72,9,60


## Write to .csv

In [79]:
df.to_csv('season_2022_w3.csv', index=False)

## List of home_teams

In [119]:
url = 'https://www.pro-football-reference.com/boxscores/202210020nor.htm'
response = requests.get(url)

In [120]:
response

<Response [200]>

In [87]:
soup = BeautifulSoup(response.content, 'html.parser')

In [106]:
href = soup.find_all('div', class_='scorebox')[0].find_all('a')[8]

In [107]:
href

<a href="/teams/nor/2022.htm">New Orleans Saints</a>

In [109]:
href.text

'New Orleans Saints'

In [110]:
team = href.text

In [83]:
response

<Response [200]>

In [75]:
home_teams = []
for game in games:
    url = f'https://www.pro-football-reference.com/{game}'
    response = requests.get(url)
    wx_digits = r'\d{1,2}'
    soup = BeautifulSoup(response.content, 'html.parser')
    div_grid = soup.find_all('div', class_='content_grid')[0]
    div = div_grid.find_all('div')[0]
    div_wrap = div.find_all('div')[0]
    soup_wrap4 = div_wrap.contents[4]
    soup4 = BeautifulSoup(soup_wrap4, 'html.parser')
    wx = soup4.find_all('td','center')[-3]
    content = str(wx.contents[0])
    hum = re.findall(wx_digits,content)[1]
    hums.append(hum)

In [76]:
hums

['54',
 '70',
 '68',
 '22',
 '63',
 '22',
 '61',
 '62',
 '72',
 '60',
 '22',
 '22',
 '67',
 '58',
 '21',
 '44']

In [113]:
wk = [i for i in range(1,23)]

In [114]:
wk

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]